In [8]:
import random
from torchvision import datasets, transforms
import torch.optim as optim
import torch.nn as nn
import pandas as pd
#import imageio
import numpy as np
import torch
from torch.autograd import Variable
import torch.nn.functional as F
import torch.utils.data as Data

import matplotlib.pyplot as plt
%matplotlib inline
import math

import torchvision
import torchvision.transforms as transforms
import optuna#载入optuna优化包

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Image preprocessing modules
transform = transforms.Compose([
    transforms.Pad(4),
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32),
    transforms.ToTensor()])



# CIFAR-10 dataset
train_dataset = torchvision.datasets.CIFAR10(root='../../data/',
                                             train=True, 
                                             transform=transform,
                                             download=True)

test_dataset = torchvision.datasets.CIFAR10(root='../../data/',
                                            train=False, 
                                            transform=transforms.ToTensor())

# Data loader
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=100, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=100, 
                                          shuffle=False)

Files already downloaded and verified


In [9]:
##optuna######
###耗费时间的过程。。。。。。。。。。。。。。。
# Define a basic convolutional layer
# 定义带参数的 GELU 激活函数层
class GELUB(nn.Module): 
    def __init__(self, seqFlag, trial):
        super().__init__()
        self.sigma = trial.suggest_float(f'sigma_{seqFlag}', 0.0, 10.0)

    def forward(self, input):
        x = input * (1 + torch.erf(input / math.sqrt(2) / self.sigma)) / 2
        return x

def conv3x3(in_channels, out_channels, stride=1):
    return nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False)

# 定义ResNet模型
class ResNet(nn.Module):
    def __init__(self, trial, num_classes=10):
        super(ResNet, self).__init__()
        self.conv1 = conv3x3(3, 16)
        self.bn1 = nn.BatchNorm2d(16)
        self.custom1 = GELUB(1, trial)  # 自定义激活函数层

        # Layer 1
        self.layer1_conv1 = conv3x3(16, 16)
        self.layer1_bn1 = nn.BatchNorm2d(16)
        self.layer1_conv2 = conv3x3(16, 16)
        self.layer1_bn2 = nn.BatchNorm2d(16)
        self.layer1_extra_conv1 = conv3x3(16, 16)
        self.layer1_extra_bn1 = nn.BatchNorm2d(16)
        self.layer1_extra_conv2 = conv3x3(16, 16)
        self.layer1_extra_bn2 = nn.BatchNorm2d(16)
        self.gelub11 = GELUB(2, trial)
        self.gelub12 = GELUB(3, trial)
        self.gelub13 = GELUB(4, trial)
        self.gelub14 = GELUB(5, trial)

        # Layer 2
        self.layer2_conv1 = conv3x3(16, 32, stride=2)
        self.layer2_bn1 = nn.BatchNorm2d(32)
        self.layer2_conv2 = conv3x3(32, 32)
        self.layer2_bn2 = nn.BatchNorm2d(32)
        self.layer2_extra_conv1 = conv3x3(16, 32, stride=2)  # 调整residual的通道数
        self.layer2_extra_bn1 = nn.BatchNorm2d(32)
        self.layer2_extra_conv2 = conv3x3(32, 32)
        self.layer2_extra_bn2 = nn.BatchNorm2d(32)
        self.gelub21 = GELUB(6, trial)
        self.gelub22 = GELUB(7, trial)
        self.gelub23 = GELUB(8, trial)
        self.gelub24 = GELUB(9, trial)
        self.layer2_downsample = nn.Sequential(
            nn.Conv2d(16, 32, kernel_size=1, stride=2, bias=False),
            nn.BatchNorm2d(32)
        )

        # Layer 3
        self.layer3_conv1 = conv3x3(32, 64, stride=2)
        self.layer3_bn1 = nn.BatchNorm2d(64)
        self.layer3_conv2 = conv3x3(64, 64)
        self.layer3_bn2 = nn.BatchNorm2d(64)
        self.layer3_extra_conv1 = conv3x3(32, 64, stride=2)  # 调整residual的通道数
        self.layer3_extra_bn1 = nn.BatchNorm2d(64)
        self.layer3_extra_conv2 = conv3x3(64, 64)
        self.layer3_extra_bn2 = nn.BatchNorm2d(64)
        self.gelub31 = GELUB(10, trial)
        self.gelub32 = GELUB(11, trial)
        self.gelub33 = GELUB(12, trial)
        self.gelub34 = GELUB(13, trial)

        self.layer3_downsample = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=1, stride=2, bias=False),
            nn.BatchNorm2d(64)
        )

        self.avg_pool = nn.AvgPool2d(8)
        self.fc = nn.Linear(64, num_classes)

    def forward(self, x):
        # Layer 0
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.custom1(out)

        # Layer 1
        residual = out
        out = self.layer1_conv1(out)
        out = self.layer1_bn1(out)
        out = self.gelub11(out)
        out = self.layer1_conv2(out)
        out = self.layer1_bn2(out)
        out += residual
        out = self.gelub12(out)
        
        residual = out
        out = self.layer1_extra_conv1(out)
        out = self.layer1_extra_bn1(out)
        out = self.gelub13(out)
        out = self.layer1_extra_conv2(out)
        out = self.layer1_extra_bn2(out)
        out += residual
        out = self.gelub14(out)

        # Layer 2
        residual = out
        out = self.layer2_conv1(out)
        out = self.layer2_bn1(out)
        out = self.gelub21(out)
        out = self.layer2_conv2(out)
        out = self.layer2_bn2(out)
        out = self.gelub22(out)
        residual = self.layer2_downsample(residual)
        out += residual
        out = self.gelub22(out)

        residual = out
        out = self.layer2_extra_conv2(out)
        out = self.layer2_extra_bn2(out)
        out = self.gelub23(out)
        out = self.layer2_extra_conv2(out)
        out = self.layer2_extra_bn2(out)
        out += residual
        out = self.gelub24(out)

        # Layer 3
        residual = out
        out = self.layer3_conv1(out)
        out = self.layer3_bn1(out)
        out = self.gelub31(out)
        out = self.layer3_conv2(out)
        out = self.layer3_bn2(out)
        residual = self.layer3_downsample(residual)
        out += residual
        out = self.gelub32(out)
        
        residual = out
        out = self.layer3_extra_conv2(out)
        out = self.layer3_extra_bn2(out)
        out = self.gelub33(out)
        out = self.layer3_extra_conv2(out)
        out = self.layer3_extra_bn2(out)
        out += residual
        out = self.gelub34(out)

        out = self.avg_pool(out)
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out

In [10]:
def update_lr(optimizer, lr):    
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

learning_rate = 0.001    
num_epochs = 80
# 定义目标函数
def objective(trial):
    # 模型实例化
    model = ResNet(trial)
    model.to(device)
    curr_lr = learning_rate
    # 损失函数和优化器
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=curr_lr)  # 保持 Adam 优化器

    #history = {'train_loss': [], 'valid_loss': [], 'train_acc': [], 'valid_acc': []}

    for epoch in range(num_epochs):
        optimizer = torch.optim.Adam(model.parameters(), lr=curr_lr)  # 保持 Adam 优化器
        train_loss, train_correct = 0.0, 0
        for i, (images, labels) in enumerate(train_loader):
            images = images.to(device)
            labels = labels.to(device)

            # Forward pass
            outputs = model(images)
            loss = criterion(outputs, labels)

            # Backward and optimize
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            train_loss += loss.item() * images.size(0)
            scores, predictions = torch.max(outputs.data, 1)
            train_correct += (predictions == labels).sum().item()

        # Decay learning rate
        if (epoch+1) % 20 == 0:
            curr_lr /= 3
            update_lr(optimizer, curr_lr)

        valid_loss, valid_correct = 0.0, 0
        model.eval()
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            valid_loss += loss.item() * images.size(0)
            scores, predictions = torch.max(outputs.data, 1)
            valid_correct += (predictions == labels).sum().item()
        ##不好的trial剪枝    
        if trial.should_prune():
                raise optuna.exceptions.TrialPruned()
            
        train_loss = train_loss / len(train_loader.sampler)
        train_acc = train_correct / len(train_loader.sampler) * 100
        valid_loss = valid_loss / len(test_loader.sampler)
        valid_acc = valid_correct / len(test_loader.sampler) * 100

        print("Epoch:{}/{} \t Train Loss:{:.4f} Valid Loss:{:.4f} \t Train Acc:{:.2f} %  Valid Acc:{:.2f} %".format(epoch + 1, num_epochs,
                                                                                         train_loss,
                                                                                         valid_loss,
                                                                                         train_acc,
                                                                                         valid_acc))
        # history['train_loss'].append(train_loss)
        # history['valid_loss'].append(valid_loss)
        # history['train_acc'].append(train_acc)
        # history['valid_acc'].append(valid_acc)

    return valid_acc

    
study = optuna.create_study(direction='maximize')

# 运行优化过程
study.optimize(objective, n_trials=2)

# 打印最佳参数和目标值
print('Best trial:')
print('  Value: ', study.best_trial.value)
print('  Params: ')
for key, value in study.best_trial.params.items():
    print('    {}: {}'.format(key, value))


[I 2024-04-21 16:01:46,323] A new study created in memory with name: no-name-8f32d8d0-c0d9-4e58-8549-7cf7d904e27c


Epoch:1/80 	 Train Loss:1.4908 Valid Loss:1.2611 	 Train Acc:45.18 %  Valid Acc:54.42 %
Epoch:2/80 	 Train Loss:1.1271 Valid Loss:1.0191 	 Train Acc:59.35 %  Valid Acc:63.37 %
Epoch:3/80 	 Train Loss:0.9919 Valid Loss:0.9680 	 Train Acc:64.68 %  Valid Acc:65.38 %
Epoch:4/80 	 Train Loss:0.8899 Valid Loss:0.8582 	 Train Acc:68.35 %  Valid Acc:69.74 %
Epoch:5/80 	 Train Loss:0.8134 Valid Loss:0.7824 	 Train Acc:71.34 %  Valid Acc:73.45 %
Epoch:6/80 	 Train Loss:0.7480 Valid Loss:0.6942 	 Train Acc:73.80 %  Valid Acc:75.98 %
Epoch:7/80 	 Train Loss:0.6894 Valid Loss:0.6576 	 Train Acc:75.96 %  Valid Acc:77.29 %
Epoch:8/80 	 Train Loss:0.6486 Valid Loss:0.6673 	 Train Acc:77.44 %  Valid Acc:77.50 %
Epoch:9/80 	 Train Loss:0.6147 Valid Loss:0.6558 	 Train Acc:78.47 %  Valid Acc:77.78 %
Epoch:10/80 	 Train Loss:0.5753 Valid Loss:0.5937 	 Train Acc:80.00 %  Valid Acc:79.70 %
Epoch:11/80 	 Train Loss:0.5457 Valid Loss:0.6292 	 Train Acc:81.05 %  Valid Acc:78.80 %
Epoch:12/80 	 Train Loss:0.527

[I 2024-04-21 16:52:08,556] Trial 0 finished with value: 87.37 and parameters: {'sigma_1': 4.446965845522028, 'sigma_2': 2.5738228033584543, 'sigma_3': 5.484707396012755, 'sigma_4': 7.447966654690194, 'sigma_5': 8.198999970169526, 'sigma_6': 0.24587717173912949, 'sigma_7': 8.339356211290225, 'sigma_8': 5.184410352497476, 'sigma_9': 1.3588377440310206, 'sigma_10': 3.2682422692579918, 'sigma_11': 5.9506674592661115, 'sigma_12': 1.0630871083712756, 'sigma_13': 6.803571010214274}. Best is trial 0 with value: 87.37.


Epoch:80/80 	 Train Loss:0.1501 Valid Loss:0.4671 	 Train Acc:94.70 %  Valid Acc:87.37 %
Epoch:1/80 	 Train Loss:1.5713 Valid Loss:1.5430 	 Train Acc:41.68 %  Valid Acc:45.56 %
Epoch:2/80 	 Train Loss:1.2719 Valid Loss:1.1742 	 Train Acc:54.08 %  Valid Acc:57.78 %
Epoch:3/80 	 Train Loss:1.1121 Valid Loss:1.0449 	 Train Acc:59.96 %  Valid Acc:62.89 %
Epoch:4/80 	 Train Loss:1.0118 Valid Loss:0.9191 	 Train Acc:64.12 %  Valid Acc:67.13 %
Epoch:5/80 	 Train Loss:0.9109 Valid Loss:0.8583 	 Train Acc:67.83 %  Valid Acc:69.46 %
Epoch:6/80 	 Train Loss:0.8365 Valid Loss:0.7877 	 Train Acc:70.43 %  Valid Acc:72.52 %
Epoch:7/80 	 Train Loss:0.7682 Valid Loss:0.7670 	 Train Acc:73.17 %  Valid Acc:73.55 %
Epoch:8/80 	 Train Loss:0.7238 Valid Loss:0.7812 	 Train Acc:74.60 %  Valid Acc:73.13 %
Epoch:9/80 	 Train Loss:0.6763 Valid Loss:0.7040 	 Train Acc:76.48 %  Valid Acc:75.62 %
Epoch:10/80 	 Train Loss:0.6392 Valid Loss:0.6622 	 Train Acc:77.76 %  Valid Acc:77.16 %
Epoch:11/80 	 Train Loss:0.602

[I 2024-04-21 18:32:17,893] Trial 1 finished with value: 86.88 and parameters: {'sigma_1': 0.5340945669600128, 'sigma_2': 2.012561708392316, 'sigma_3': 0.03452267338606485, 'sigma_4': 7.165112415675016, 'sigma_5': 5.710022096981993, 'sigma_6': 7.585482192543815, 'sigma_7': 9.745801957430501, 'sigma_8': 3.625283891054244, 'sigma_9': 6.441710761200471, 'sigma_10': 2.9233715401612925, 'sigma_11': 4.722315315920089, 'sigma_12': 6.307206064673162, 'sigma_13': 1.1530864499956828}. Best is trial 0 with value: 87.37.


Epoch:80/80 	 Train Loss:0.1722 Valid Loss:0.4852 	 Train Acc:93.87 %  Valid Acc:86.88 %
Best trial:
  Value:  87.37
  Params: 
    sigma_1: 4.446965845522028
    sigma_2: 2.5738228033584543
    sigma_3: 5.484707396012755
    sigma_4: 7.447966654690194
    sigma_5: 8.198999970169526
    sigma_6: 0.24587717173912949
    sigma_7: 8.339356211290225
    sigma_8: 5.184410352497476
    sigma_9: 1.3588377440310206
    sigma_10: 3.2682422692579918
    sigma_11: 5.9506674592661115
    sigma_12: 1.0630871083712756
    sigma_13: 6.803571010214274
